In [1]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mgimg
import scipy as sp
import numpy as np
from scipy import interpolate
from scipy.interpolate import interp1d
from IPython.display import display
%matplotlib inline
#%config InlineBackend.close_figures = False
from scipy.io.idl import readsav


In [2]:
rawfilt = np.loadtxt("UKIRT-UKIDSS.J.dat",unpack="True",skiprows=0)
# load spex spectrum and skip to 1um
rawspec = np.loadtxt("spex-prism_Gliese570D_20030522_BUR04B.txt",unpack="True",skiprows=138)
rawvega = np.loadtxt("vega.txt", unpack="True", skiprows=6)
print rawfilt.shape
rawfilt[0,:] = rawfilt[0,:] / 10000.
rawvega[0,:] = rawvega[0,:] / 10000.
rawvega[1,:] = rawvega[1,:] * 10. # erg/cm/s/A to W/m2/um

(2, 409)


In [3]:
# Now trim target spectrum to wavelength region of filter, and rebin filter onto new target grid
w1 = rawfilt[0,0]
w2 = rawfilt[0,rawfilt.shape[1] -1]
trimspec =  rawspec[:,np.logical_not(np.logical_or(rawspec[0,:] > w2, rawspec[0,:] < w1))]
wfit = sp.interpolate.splrep(rawfilt[0,:],rawfilt[1,:],s=0)
filt = np.zeros_like(trimspec)
filt[0,:] = trimspec[0,:]
# multiply filter by lambda and renormalise 
wfilt = sp.interpolate.splev(filt[0,:],wfit,der=0)# * filt[0,:]
filt[1,:] = wfilt / np.amax(wfilt)
# and vega
trimvega =  rawvega[:,np.logical_not(np.logical_or(rawvega[0,:] > w2, rawvega[0,:] < w1))]
wfit = sp.interpolate.splrep(trimvega[0,:],trimvega[1,:],s=0)
vega = np.zeros_like(trimspec)
vega[0,:] = filt[0,:]
vega[1,:] = sp.interpolate.splev(filt[0,:],wfit,der=0)

In [4]:
Mag = 14.82
intspec = sum(filt[1,:]*trimspec[1,:])
intvega = sum(filt[1,:]*vega[1,:])
scale =(10.**(-Mag/2.5)) * (intvega / intspec)
print scale

5.92277552972e-15


In [5]:
scale = 5.9e-15
scalespec = np.empty_like(rawspec)
scalespec[0,:] = rawspec[0,:]
scalespec[1,:] = rawspec[1,:] * scale
scalespec[2,:] = rawspec[2,:] * scale

In [6]:
np.savetxt('G570D_calib.dat', np.c_[scalespec[0],scalespec[1],scalespec[2]])